（※このファイルはGoogle Colaboratory用です。他の環境では未検証です）

（※**Googleドライブの機能を使用します。**途中の結果がドライブ上に保存されます。空き容量に注意してください）

（※2-1.以降の実行後、再度1-1.から実行したい場合は「ランタイム→すべてのランタイムのリセット...」を実行してリセットしてください）

##1. 音源の学習処理

1-1. 以下のコードを実行し、音声ファイルを圧縮したzipファイルをアップロードしてください。

In [0]:
%cd /content

# 音声の入ったzipファイルをアップロード
from google.colab import files
import os

uploaded = files.upload()

for fn in uploaded.keys():
    os.rename(fn, 'targets.zip')

1-2. メイン処理です。以下のコードを実行してください。

（途中でURLが表示され、止まることがあります（合計2回）。その場合はURLにアクセスし、キーを入力欄に貼り付けてください）

（タイムアウトなどで途中で終了した場合でも、このコードを再度実行すれば続きから再開されます）

（一度、他の音源を作成した場合は、実行前にドライブ上のnvzディレクトリを移動させるか、retrain_flagをFalseにしてください）

In [0]:
# ↓途中で終了した場合にそこから再開するか。新規に作成する場合はFalseにすること
retrain_flag = True
# ---

import zipfile
import os
import shutil

%cd /content

if not os.path.isdir('NVC_train'):
    # リポジトリをクローンし、フォルダ内へ移動
    !git clone https://github.com/NON906/NVC_train.git
    %cd NVC_train

    # Googleドライブに接続
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
    !apt-get update -qq 2>&1 > /dev/null
    !apt-get -y install -qq google-drive-ocamlfuse fuse

    from google.colab import auth
    auth.authenticate_user()

    from oauth2client.client import GoogleCredentials
    creds = GoogleCredentials.get_application_default()
    import getpass
    !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
    vcode = getpass.getpass()
    !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

    !mkdir -p drive
    !google-drive-ocamlfuse drive
else:
    %cd NVC_train
    
# メイン処理開始
from scripts.make_nvm import make_nvm_main
from scripts.gen_targets import gen_targets_main
from scripts.target_train import target_train_main
from scripts.target_pitch_train import target_pitch_train_main

outputs_dir = os.getcwd() + '/drive/nvc'

retrain_step = 0
if retrain_flag and os.path.isfile(outputs_dir + '/retrain.dat'):
    retrain_step = int(open(outputs_dir + '/retrain.dat', 'r').read())
else:
    if os.path.isdir(outputs_dir):
        shutil.rmtree(outputs_dir)
    if os.path.isdir('gen_targets'):
        shutil.rmtree('gen_targets')

if retrain_step < 2:
    # soxのインストール
    !apt-get -y install sox

    # SPTKのインストール
    !apt-get -y install csh
    !wget http://downloads.sourceforge.net/sp-tk/SPTK-3.11.tar.gz
    !tar xfvz SPTK-3.11.tar.gz
    %cd SPTK-3.11
    !./configure
    !make
    !make install
    %cd ..

    # アップロードしたzipファイルを解凍
    !apt-get install unar
    !unar -o targets ../targets.zip

if retrain_step < 1:
    make_nvm_main('targets', outputs_dir + '/target.nvm')
    open(outputs_dir + '/retrain.dat', 'w').write('1')
    
if retrain_step < 2:
    gen_targets_main('targets', 'gen_targets', outputs_dir + '/gen_targets.zip')
    open(outputs_dir + '/retrain.dat', 'w').write('2')
else:
    with zipfile.ZipFile(outputs_dir + '/gen_targets.zip') as existing_zip:
        existing_zip.extractall('gen_targets')

if retrain_step < 3:
    if not os.path.isfile(outputs_dir + '/target.h5'):
        target_train_main('gen_targets', outputs_dir + '/target.h5', 20, -1, 32)
    else:
        target_train_main('gen_targets', outputs_dir + '/target.h5', 20, -1, 32, outputs_dir + '/target.h5')
    open(outputs_dir + '/retrain.dat', 'w').write('3')

if retrain_step < 4:
    if not os.path.isfile(outputs_dir + '/pitch.h5'):
        target_pitch_train_main('gen_targets', outputs_dir + '/pitch.h5', 20, -1, 32)
    else:
        target_pitch_train_main('gen_targets', outputs_dir + '/pitch.h5', 20, -1, 32, outputs_dir + '/pitch.h5')
    open(outputs_dir + '/retrain.dat', 'w').write('4')
else:
    print('ドライブ上のnvzディレクトリを移動させるか、retrain_flagをFalseにしてください')

---

##2. 学習データの変換

2-1. 以下のコードを実行してください。

In [0]:
!pip install tensorflow==1.7.1

2-2. 上部メニューの「ランタイム→ランタイムを再起動...」から再起動してください。

2-3. 以下のコードを実行してください。

In [0]:
import sys
import os

%cd /content

if not os.path.isdir('NVC_train'):
    # リポジトリをクローンし、フォルダ内へ移動
    !git clone https://github.com/NON906/NVC_train.git
    %cd NVC_train

    # Googleドライブに接続
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
    !apt-get update -qq 2>&1 > /dev/null
    !apt-get -y install -qq google-drive-ocamlfuse fuse

    from google.colab import auth
    auth.authenticate_user()

    from oauth2client.client import GoogleCredentials
    creds = GoogleCredentials.get_application_default()
    import getpass
    !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
    vcode = getpass.getpass()
    !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

    !mkdir -p drive
    !google-drive-ocamlfuse drive
else:
    %cd NVC_train
    
from scripts.target_convert_h5_to_pb import convert_h5_to_pb_main
from scripts.make_nvz import make_nvz_main

outputs_dir = os.getcwd() + '/drive/nvc'

convert_h5_to_pb_main(outputs_dir + '/target.h5', outputs_dir + '/target.pb', 'target_dense_f/add')
convert_h5_to_pb_main(outputs_dir + '/pitch.h5', outputs_dir + '/pitch.pb', 'pitch_relu_f/Relu')

make_nvz_main(outputs_dir + '/target.nvz', outputs_dir + '/target.nvm', outputs_dir + '/target.pb', outputs_dir + '/pitch.pb')

note. 2-3.の実行後、以下のコードからダウンロード版用のファイルをダウンロードできます。

In [0]:
%cd /content/NVC_train

from google.colab import files

outputs_dir = os.getcwd() + '/drive/nvc'
files.download(outputs_dir + '/target.nvz')

---

##3. ブラウザ版用のデータ生成＆アップロード

3-1. 以下のコードを実行してください。

In [0]:
!pip install tensorflowjs

3-2. 上部メニューの「ランタイム→ランタイムを再起動...」から再起動してください。

3-3. 以下のコードを実行してください。

In [0]:
import os
import sys
import requests

%cd /content/NVC_train

outputs_dir = os.getcwd() + '/drive/nvc'

!tensorflowjs_converter --input_format=tf_frozen_model --output_node_names="target_dense_f/add" "{outputs_dir}/target.pb" "{outputs_dir}/target/web_model"
!tensorflowjs_converter --input_format=tf_frozen_model --output_node_names="pitch_relu_f/Relu" "{outputs_dir}/pitch.pb" "{outputs_dir}/pitch/web_model"

url = 'https://www.mumeigames.online/nvc/php/upload.php'
files = {
    'target_pb': open(outputs_dir + '/target/web_model/tensorflowjs_model.pb', 'rb'),
    'target_value': open(outputs_dir + '/target/web_model/group1-shard1of1', 'rb'),
    'target_json': open(outputs_dir + '/target/web_model/weights_manifest.json', 'rb'),
    'pitch_pb': open(outputs_dir + '/pitch/web_model/tensorflowjs_model.pb', 'rb'),
    'pitch_value': open(outputs_dir + '/pitch/web_model/group1-shard1of1', 'rb'),
    'pitch_json': open(outputs_dir + '/pitch/web_model/weights_manifest.json', 'rb'),
    'nvm': open(outputs_dir + '/target.nvm', 'rb'),
    'nvz': open(outputs_dir + '/target.nvz', 'rb'),
}
res = requests.post(url, files=files)
if res.status_code != requests.codes.ok:
    sys.exit()
result_id = res.text
if result_id == '':
    sys.exit()
print('---')
print('音源データの生成・アップロードが完了しました。')
print('ID: ' + result_id)
print('---')
print('実行用URL: https://www.mumeigames.online/nvc/?id=' + result_id)
print('ツイートして他の人に伝える: https://twitter.com/?status=%e3%80%8c%e3%80%8d%e3%81%ab%e3%81%aa%e3%82%8a%e3%81%8d%e3%82%8b%e3%81%9f%e3%82%81%e3%81%ae%e3%83%9c%e3%82%a4%e3%82%b9%e3%83%81%e3%82%a7%e3%83%b3%e3%82%b8%e3%83%a3%e3%83%bc%e3%82%92%e4%bd%9c%e3%81%a3%e3%81%a6%e3%81%bf%e3%81%9f%ef%bc%9ahttps%3a%2f%2fwww%2emumeigames%2eonline%2fnvc%2f%3fid%3d' + result_id)
print('---')